# Using Python Get Data From Baidu API

In [1]:
#coding=utf-8
import json
import geopandas
import matplotlib.pyplot as plt
import requests
from shapely.geometry import LineString
import time
import os

In [2]:
BASE_URL = 'https://restapi.amap.com/v3/traffic/status/rectangle?rectangle={loc}&key={k}&extensions=all'
keys = [
    '0f77fda9a61b3fa8ae32f99b2eabe534', 'ac644665b36b3ca24b4a7abe54661efb',
    'd718d4a3addce3439b3df02c0e8b599c',"2b239cb466670c8379747913be5ba79e"
]
Loc_of_Shanghai = '121.473658,31.230378'
key_flag = 3
Lines = []
Names = []
Speeds = []
Status = []
Lenghts = []
Lcodes = []
Timestamps = []

In [3]:
class Grid(object):
    def __init__(self, loc_all):
        self.loc_all = loc_all
        self.lng_sw = float(self.loc_all.split(',')[0])
        self.lat_sw = float(self.loc_all.split(',')[1])
        self.max = 0.06

    def get_max(self):
        for i in range(1, 10):
            data = str(self.lng_sw) + "," + str(self.lat_sw) + ';' + str(
                self.lng_sw + i / 100) + ',' + str(self.lat_sw + i / 100)
            print(i)
            url = BASE_URL.format(loc=data, k=keys[key_flag])
            print(url)
            res = requests.get(url)
            datas = res.json()
            if datas['status'] != '1':
                print(
                    "the maxium of distance is",
                    i,
                )
                break

    def get_grid(self):
        grid = []
        lng = []
        lat = []
        for i in range(-5,3):
            lng.append(self.lng_sw + self.max * i)
            lat.append(self.lat_sw + self.max * i)
        for i in range(8):
            for j in range(8):
                data = str(lng[i])+','+str(lat[j])+';'+str(lng[i]+self.max)+','+str(lat[j]+self.max)
                grid.append(data)
        return grid

In [4]:
loc = Grid(Loc_of_Shanghai)
locs= loc.get_grid()

In [5]:
#coding=utf-8

def request_map_one_minute():
    Times = time.strftime('%Y_%m_%d %H:%M:%S',time.localtime())
    global Lines, Names, Speeds, Status, Lenghts, Lcodes, Timestamps, locs, key
#     all_datas = []
    for i in locs:
        url = BASE_URL.format(loc=i, k=keys[3])
#         print(url)
        res = requests.get(url)
        datas = res.json()
#         all_datas.append(datas)
        if datas['status'] == '1':
            for road in datas['trafficinfo']['roads']:
                polyline = road['polyline']
                polylines = [
                    (float(y[0]), float(y[1])) for y in
                    [x.split(',') for x in [i for i in polyline.split(';')]]
                ]
                line = LineString(polylines)
                Lines.append(line)
                Names.append(road['name'])
                Lcodes.append(road['lcodes'])
                Timestamps.append(Times)
                if 'speed' in road: Speeds.append(int(road['speed']))
                else: Speeds.append(100)
                Status.append(road['status'])
                Lenghts.append(line.length)
        else:
            if datas['infocode'] == '10003':
                key_flag = (key_flag + 1)%len(keys)
            if datas['infocode'] == '20003':
                continue
            else:
                print("Error of status",datas['infocode'])
#     with open("record.json","w") as f:
#         json.dump(all_datas,f)
#         print("加载入文件完成...")


def plot_show(pds_dict):
    ShangHai = geopandas.GeoDataFrame(pds_dict)
    #     ShangHai.to_file("shanghai.shp")
    #     ShangHai = geopandas.read_file("shanghai.shp")
    ShangHai.plot(column='speed', cmap='RdYlGn', legend=True)
    plt.figure(dpi=400)
    plt.show()


def save_to_file(data):
    datas = geopandas.GeoDataFrame(data)
    ti = "Shanghai_" + str(
        str(time.localtime()[1]) + '_' + str(time.localtime()[2]) + '_' +
        str(time.localtime()[3]))
    datas.to_file(ti,encoding='utf-8')

In [ ]:
def print_ts(message):
    print ("[%s] %s"%(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), message))

def run(interval, command):
    print_ts("Starting every %s seconds during 6:00 -- 21:00."%interval)
    global Lines,Names,Speeds,Status,Lenghts,Lcodes,Timestamps,locs,key
    Start_hour =10
    while True:
        T_hour = time.localtime()[3]
        if T_hour >= 6 and T_hour <= 21:
            if T_hour != Start_hour:
                datas = {'name':Names,'Timestamp':Timestamps,'Lcode':Lcodes,"speed":Speeds,'CongestionRate':Status,'lenght':Lenghts,"geometry":Lines}
                save_to_file(datas)
                print_ts("Finish Save to file")
                Start_hour = T_hour
                Lines.clear()
                Names.clear()
                Speeds.clear()
                Status.clear()
                Lenghts.clear()
                Lcodes.clear()
                Timestamps.clear()
            # sleep for the remaining seconds of interval
            # execute the command
            time_remaining = interval-time.time()%interval
            print_ts("Sleeping until %s (%s seconds)..."%((time.ctime(time.time()+time_remaining)), time_remaining))
            time.sleep(time_remaining)
            try :
                request_map_one_minute()
            except:
                print("Error")
            print_ts("Starting command.")
            print_ts("-"*100)
        else:
            time.sleep(time_remaining*command)
            
if __name__=="__main__":
    interval = 120
    command  = 100
    run(interval, command)

[2019-11-21 10:16:14] Starting every 120 seconds during 6:00 -- 21:00.
[2019-11-21 10:16:14] Sleeping until Thu Nov 21 10:18:00 2019 (105.59201693534851 seconds)...
[2019-11-21 10:18:08] Starting command.
[2019-11-21 10:18:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 10:18:08] Sleeping until Thu Nov 21 10:20:00 2019 (111.82499504089355 seconds)...
[2019-11-21 10:20:09] Starting command.
[2019-11-21 10:20:09] ----------------------------------------------------------------------------------------------------
[2019-11-21 10:20:09] Sleeping until Thu Nov 21 10:22:00 2019 (110.963534116745 seconds)...
[2019-11-21 10:22:08] Starting command.
[2019-11-21 10:22:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 10:22:08] Sleeping until Thu Nov 21 10:24:00 2019 (111.57877612113953 seconds)...
[2019-11-21 10:24:08] Starting command.
[2019-11-21 10:24:08] ----

[2019-11-21 11:22:08] Starting command.
[2019-11-21 11:22:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 11:22:08] Sleeping until Thu Nov 21 11:24:00 2019 (111.33823490142822 seconds)...
[2019-11-21 11:24:08] Starting command.
[2019-11-21 11:24:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 11:24:08] Sleeping until Thu Nov 21 11:26:00 2019 (111.95109701156616 seconds)...
[2019-11-21 11:26:08] Starting command.
[2019-11-21 11:26:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 11:26:08] Sleeping until Thu Nov 21 11:28:00 2019 (111.85940599441528 seconds)...
[2019-11-21 11:28:17] Starting command.
[2019-11-21 11:28:17] ----------------------------------------------------------------------------------------------------
[2019-11-21 11:28:17] Sleeping until Thu Nov 21 11:30:00 2019 (102

[2019-11-21 12:26:09] Starting command.
[2019-11-21 12:26:09] ----------------------------------------------------------------------------------------------------
[2019-11-21 12:26:09] Sleeping until Thu Nov 21 12:28:00 2019 (110.30382204055786 seconds)...
Error of status 30003
[2019-11-21 12:28:12] Starting command.
[2019-11-21 12:28:12] ----------------------------------------------------------------------------------------------------
[2019-11-21 12:28:12] Sleeping until Thu Nov 21 12:30:00 2019 (107.66230797767639 seconds)...
[2019-11-21 12:30:08] Starting command.
[2019-11-21 12:30:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 12:30:08] Sleeping until Thu Nov 21 12:32:00 2019 (111.91672015190125 seconds)...
[2019-11-21 12:32:08] Starting command.
[2019-11-21 12:32:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 12:32:08] Sleeping until Thu Nov

In [ ]:
#coding=utf-8
request_map_one_minute()
datas = {'name':Names,'Timestamp':Timestamps,'Lcode':Lcodes,"speed":Speeds,'CongestionRate':Status,'lenght':Lenghts,"geometry":Lines}
plot_show(datas)